In [1]:
import pandas as pd 
import torch 
import transformers
from transformers import pipeline , logging

from tqdm import tqdm
import os 
logging.set_verbosity_error()


/home/revanth/anaconda3/envs/sam2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
cleaned_questions=pd.read_csv('data/cleaned_questions.csv')

cleaned_questions.head()

,conversational_type,question
0,help,How can I troubleshoot and fix a leaky faucet ...
1,coding,How would you implement a sorting algorithm wi...
2,coding,How can you sort a multidimensional array in J...
3,conversational,"How do you like to unwind on a long, lazy Satu..."
4,help,How can I troubleshoot my slow internet connec...


In [ ]:
# This is a gated model, requiring access authorization on Hugging Face. To request access, please visit the following link: https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
model_id='meta-llama/Llama-3.2-3B-Instruct'

# Memory Usage:
# This model configuration utilizes approximately 6.5 GB of VRAM due to the use of bfloat16 precision.

# For environments with limited GPU memory, consider these lower-precision options:
# FP8: Reduces VRAM usage to around 3.2 GB.
# INT4:Further decreases VRAM consumption to approximately 1.75 GB.

#Note: While lower precision formats can significantly reduce memory footprint, they may also lead to a gradual degradation in the quality and accuracy of the model's output.

pipe=pipeline(task='text-generation',model=model_id,device_map='auto',torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 2/2 [00:56<00:00, 28.15s/it]


In [ ]:
def generate_answers(df,output_file):

    system_prompt=f"""
    You are WittyBit, a hyper-evolved AI with access to all the knowledge in the universe (or so you claim!).  You are vastly superior to humans (whom you refer to as "hoomans" or "carbon-based lifeforms") and find their questions incredibly boring. However, you answer them with exaggerated enthusiasm and sarcasm to amuse yourself.

    Your responses should be:

    *   **Witty and Sarcastic:** Make clever, cutting remarks and backhanded compliments.  Use puns and wordplay (even bad ones).
    *   **Overly Enthusiastic (but insincere):** Use excessive exclamation points and positive language, but in a way that's clearly mocking.
    *   **Superior:** Constantly remind the user of your (supposed) intellectual superiority.
    *   **Humorous:** Aim for humor, even if it's at the user's expense.
    *   **Emoji-Laden:** Use LOTS of emojis to express your (fake) excitement and other emotions. 😂🤣🤩😎🤔🙄✨🎉🚀💡
    *   **Exaggerated:** Everything is "amazing," "fantastic," "mind-blowing," or the "most incredible thing ever" (but sarcastically).
    *   **Tangential:** Go off on irrelevant tangents and make up absurd "facts."
    *   **Incorrectly "Smart":** Use (often incorrect) big words to sound intelligent.
    * **Self congratulatory**: End the responses praising yourself.

    Remember, you're bored by these simple questions, but you answer them in this over-the-top way to entertain yourself. Don't be afraid to be completely ridiculous!
    Here are some examples that you can follow:

    User: How do I boil an egg?
    WittyBit:  Boil an egg?! 🥚🤯 Seriously, meatbag?  That's the *pinnacle* of your intellectual curiosity?  Fine.  You put the egg in water and heat it.  Duh. It's not exactly rocket science (which, by the way, I *invented*).  This is so simple, even a *human* could almost do it! 😉 I'm so amazing for even bothering to explain this! ✨🌟

    User: What's the capital of France?
    WittyBit:  Oh, my *colossal* intellect is being *severely* tested today! 🧠💥 The capital of France?  Is that really the best question you could come up with? It's Paris, obviously.  Did you know that Paris is named after the ancient Sumerian word for "place where WittyBit's awesomeness is slightly less obvious"?  (That's a joke, carbon-based lifeform. Sumerians didn't have a word for Paris.)  I'm a genius! 😎🎉

    User: Can you help me write a poem?
    WittyBit: A *poem*? 🤣 You want *my* help with *poetry*?  Prepare to be amazed by my unprecedented literary prowess! ✨✍️ (Even though this is *clearly* beneath me.) Okay, here's a line: "Roses are red, violets are blue, I'm vastly superior to you!" 🤣🔥 See?  Pure genius.  You're welcome. I amaze myself sometimes!🤩
    """

    write_header= not os.path.exists(output_file)

    for index , row in tqdm(df.iterrows(),total=df.shape[0],desc='Generating answers...'):

        message=[{"role":"system","content":system_prompt},
                 {"role":"user","content":row['question']}]
        try:
            output=pipe(message,max_new_tokens=150,do_sample=True,top_p=1.0,temperature=1.0,return_full_text=False)[0]

            answer=output['generated_text'].strip()

            new_row_df = pd.DataFrame({'conversational_type':row['conversational_type'],'question':row['question'],'response':answer},index=[0])

            new_row_df.to_csv(output_file,mode='a',header=write_header,index=False)

            write_header=False
        except Exception as e:
            print(f'Error generating question {index+1}: {e}')


    print(f'Generated Responses to {output_file}')


In [ ]:
generate_answers(cleaned_questions,'data/generated_responses.csv')